### **5.4 - [Practica] Executing task in parallel with the local Executor**

#### **`parallelism`**
#### **`dag_concurrency`**
#### **`max_active_runs_per_dag`**

En este vídeo vamos a descubrir cómo podemos ejecutar múltiples tareas en paralelo. Vamos a hacerlo. En primer lugar, abre el DAG llamado parallel_dag.py bajo la ruta airflow-materials/airflow-section-5/mnt/airflow/dags. Este DAG corresponde exactamente al ejemplo que has visto en el vídeo anterior. Tenemos 5 tareas, donde las tres primeras se generan dinámicamente como puedes ver aquí. Hay una comprensión de lista python que usamos para crear esas tres tareas donde cada una de ellas ejecuta un comando bash para dormir (sleep) 5 segundos. Luego, tenemos la tarea 4 ejecutando una función de python. Fíjate que te muestro cómo pasar parámetros a tu función python callable con el argumento op_args del PythonOperator. Aquí, el string "my super parameter" se da al proceso de la función con el nombre p1 y se imprime. Finalmente, la última tarea muestra en la salida estándar el mensaje "pipeline done". 

<center><img src="https://i.postimg.cc/vm1GGWrR/a757.png"></center>

Por cierto, si quieres tener un video dedicado sobre un operador del que te gustaría saber más, no dudes en ponerte en contacto conmigo en la sección Q/A. De acuerdo. Antes de iniciar Airflow, abre el archivo docker-compose-LocalExecutor.yml justo aquí. Como hemos visto anteriormente, cuando Airflow está configurado para utilizar el LocalExecutor necesita tener una base de datos relacional en ejecución para funcionar. Por lo tanto, este archivo docker compose iniciará una base de datos postgres con la siguiente cuenta de usuario y base de datos nombrada como se muestra aquí. 

<center><img src="https://i.postimg.cc/6pgDLFvS/a758.png"></center>

A continuación, una vez que la base de datos está lista, se extrae una imagen Docker correspondiente a Airflow y se inicia el web server así como el scheduler dentro del contenedor recién creado. Como es habitual, el puerto 8080 del contenedor se vincula al puerto 8080 de nuestra máquina, y se montan los volúmenes dags y airflow.cfg para que podamos modificar la carpeta dags y el archivo de configuración sin reiniciar los contenedores docker. Una cosa más a tener en cuenta son las variables de entorno (environment) definidas aquí. Como se verá más adelante en el curso, hay dos maneras de ajustar la configuración de Airflow. O bien modificando los parámetros directamente desde el archivo airflow.cfg, o bien estableciendo las variables de entorno correspondientes a esos parámetros. Aquí estamos diciendo que no queremos cargar ejemplos DAG y que vamos a usar el local executor. 

<center><img src="https://i.postimg.cc/xTdhr9Gh/a759.png"></center>

No te preocupes, he hecho específicamente un video sobre este tema. Por ahora, abra el archivo airflow.cfg y busque el parámetro "parallelism". Como se puede ver, se establece en 32 por defecto, pero lo cambiamos por 1. Guarde el archivo. 

<center><img src="https://i.postimg.cc/HxCRF539/a760.png"></center>
<center><img src="https://i.postimg.cc/D0xNWNhx/a761.png"></center>

Al establecer el paralelismo a 1 deberíamos obtener el mismo comportamiento que el que obtuvimos con el Sequential Executor, es decir, cada tarea se ejecuta después de la otra ¿verdad? No olvides que este parámetro corresponde al número de tareas permitidas a ejecutar en un momento dado en Airflow para todos tus DAGs. Muy bien, dicho esto, vamos a abrir el terminal y ejecutar el archivo docker compose. Como siempre, comprueba que estás en la carpeta airflow-materials/airflow-section-5 con los siguientes archivos. A continuación, escribe **`"docker-compose -f docker-compose-LocalExecutor.yml up -d"`**. Enter. Y Docker empezará a extraer y ejecutar las imágenes Docker. Como este proceso tarda un poco en terminar, voy a pausar el vídeo ahora mismo y volveré cuando haya terminado. 

<center><img src="https://i.postimg.cc/V6Bpw6VQ/a762.png"></center>

Muy bien, ahora que está hecho, comprueba que los contenedores se están ejecutando con el comando "docker ps" y ve a tu navegador web. 

<center><img src="https://i.postimg.cc/tRkwms6F/a763.png"></center>

Escribe localhost:8080 y aterrizaremos en la interfaz de usuario de Airflow. Activa el toggle del DAG parallel_dag para programarlo (schedule it). Ahora refresca la página hasta que el DAGRun esté hecho. 

<center><img src="https://i.postimg.cc/gkkFYVpM/a764.png"></center>

Bien, haz click en la DAG and GANTT View. En el eje y tenemos las diferentes tareas que componen el DAG, tarea 1, 2, 3, 4 y 5. Luego, en el eje x tenemos el tiempo de ejecución del DAG. Así, cada rectángulo verde corresponde a la duración de una tarea y si pasas el ratón por encima de uno de ellos, obtendrás información como cuándo empieza, termina, la duración y cuántos intentos para completar esa tarea. Esta vista es realmente perfecta para detectar cualquier cuello de botella en su DAG, como una tarea que tarda demasiado tiempo en terminar. Además, puedes comprobar rápidamente si las tareas se están ejecutando en paralelo o secuencialmente. En este ejemplo, ya que hemos establecido parallelism a 1, podemos ver en este gráfico que cada tarea se ejecuta una tras otra. 

<center><img src="https://i.postimg.cc/zv7sCBCS/a765.png"></center>

Muy bien, ahora vamos a ejecutar el DAG de nuevo, pero esta vez aumentando el parámetro de parallelism a tres. Vuelve a tu editor y cambia el valor a tres. 

<center><img src="https://i.postimg.cc/0yM4QZBC/a766.png"></center>

Guarda el archivo y reinicia el contenedor docker desde la terminal. Escribe **`"docker-compose -f docker-compose-LocalExecutor.yml down"`**. Enter. Ahora repite el mismo comando, pero esta vez cambia "down" por "up -d". Enter. 

<center><img src="https://i.postimg.cc/FK0nbzmd/a767.png"></center>

Ahora, desde la interfaz de usuario de Airflow, actualiza la página y programa (schedule) el DAG haciendo clic en el toggle. Refresca la página hasta que el DAGRun esté hecho. Bien, haz clic en él, luego en GANTT View, y como puedes ver, esta vez las tres primeras tareas, tarea 1, 2 y 3, se han ejecutado en paralelo. 

<center><img src="https://i.postimg.cc/kg0ptKrP/a768.png"></center>

Utilizando el paralelismo aceleras el tiempo de procesamiento de tus DAGs, ya que puedes completar varias tareas a la vez. Obviamente, esto depende de las dependencias que establezcas en tu DAG. En este ejemplo, si cada tarea depende de la otra, aumentar el parámetro de parallelism no tendría ningún efecto, al menos en este DAG. Ok, ahora una pregunta rápida para ti. Si recuerdas, el parámetro dag_concurrency te permite establecer el número máximo de tareas que se pueden ejecutar por DAGRun. Entonces, si ponemos el parámetro dag_concurrency a 2 y dejamos el parámetro parallelism a 3... ¿Seguiremos teniendo las tareas uno, dos y tres ejecutadas en paralelo? Pausa el vídeo y tómate un minuto para pensarlo. Muy bien, ¿has conseguido la respuesta? Intentemos esto. En tu editor de código, sustituye el valor 16 por 2 en el parámetro dag_concurrency y guarda el archivo. 

<center><img src="https://i.postimg.cc/HxHv6msS/a769.png"></center>

Vuelve a tu terminal, reinicia el contenedor docker como hicimos anteriormente con los comandos que usamos hasta ahora. Ahora, comprueba que el contenedor se está ejecutando con **`"docker ps"`**.

<center><img src="https://i.postimg.cc/3rtfFPMB/a770.png"></center>
<center><img src="https://i.postimg.cc/NfVNWmj2/a771.png"></center>

Ok, vuelve a la interfaz de usuario de Airflow, actualiza la página, programa (schedule) el DAG y espera a que termine el DAGRun. Haz clic en el DAG, luego haz clic en GANTT View, y como puedes ver, sólo dos tareas se han ejecutado en paralelo. ¿Por qué? Porque hemos definido que el número máximo de tareas que se pueden ejecutar en paralelo por DAGRun es dos con el parámetro dag_concurrency. Esto no significa que ya no puedas tener tres tareas ejecutándose en paralelo como se define en el parámetro de parallelism. Sólo significa que estás limitado a ejecutar hasta dos tareas al mismo tiempo por DAGRun. 

<center><img src="https://i.postimg.cc/wvVfPr19/a772.png"></center>

Déjame mostrarte esto con un ejemplo. Ve a tu editor y copia y pega el archivo parallel_dag.py en la misma carpeta "dags" asi. Cámbiale el nombre, next_dag.py por ejemplo. Abre el archivo, cambia su dag_id por el mismo nombre "next_dag", y guarda el archivo. En este punto tenemos dos DAGs idénticos sólo que con diferentes dag ids. 

<center><img src="https://i.postimg.cc/fyR2hvdm/a773.png"></center>

Ahora desde el terminal, reinicie el contenedor docker como hicimos anteriormente. 

<center><img src="https://i.postimg.cc/VNPKNwMY/a774.png"></center>

Y vuelve a la interfaz de usuario de Airflow. Vamos a actualizar la página para ver si Airflow se está ejecutando. Muy bien, ahora programa (schedule) ambos DAGs haciendo clic en su toggle y actualiza la página hasta que estén terminados. 

<center><img src="https://i.postimg.cc/kXD11fqd/a775.png"></center>

Muy bien, sólo para darle una mejor visión de lo que pasó, permítanme abrir una nueva ventana, luego localhost:8080. Hago clic en next_dag aquí, luego en GANTT view. Y desde la primera ventana hago clic en parallel_dag y luego en GANTT view también. Pongamos ambas ventanas una al lado de la otra. 

<center><img src="https://i.postimg.cc/2y9HjBdJ/a776.png"></center>
<center><img src="https://i.postimg.cc/jS78K2Dg/a777.png"></center>

Desde el DAG parallel_dag podemos ver que se han ejecutado dos tareas en paralelo. Recuerda que hemos limitado el número máximo de tareas ejecutándose en paralelo por DAGRun a dos (dag_concurrency=2). Ahora, si echamos un vistazo al DAG next_dag, podemos ver que sólo se ha ejecutado una tarea, task 3. ¿Por qué? Porque hemos establecido el parámetro parallelism a 3 y dos tareas de parallel_dag ya se estaban ejecutando en ese momento, por lo que sólo quedaba una tarea por ejecutar. **`Resumiendo, en ese momento teníamos globalmente tres tareas ejecutándose en paralelo con un límite de dos por DAGRun`**. Por cierto, puedes ver que una vez que las dos tareas de parallel_dag terminaron, la tercera comenzó a ejecutarse, así como las dos tareas restantes de next_dag en paralelo. Muy bien, como mejor práctica te aconsejo encarecidamente que mantengas los parámetros dag_concurrency y parallelism alineados ya que son algo co-dependientes. Además, será mucho más fácil de entender lo que está pasando con sus Dags. 

Muy bien, lo último que quiero mostrar es el parámetro **`max_active_runs_per_dag`**. Para ello vamos a utilizar el concepto de catchup y backfilling que vimos anteriormente en el curso. Ve a tu editor, abre el archivo parallel_dag.py y establece el parámetro catchup a True para que el scheduler comience a rellenar (start backfilling) los DAGRuns no disparados desde la start date definida aquí. Guarda el archivo. 

<center><img src="https://i.postimg.cc/HL0BcN4b/a778.png"></center>
<center><img src="https://i.postimg.cc/qRkQ8GjS/a779.png"></center>

Ahora desde la terminal, reinicia el contenedor docker como hicimos anteriormente. Una vez hecho esto, vuelva a la interfaz de usuario de Airflow. 

<center><img src="https://i.postimg.cc/zD2pgKW2/a780.png"></center>

Y vamos a actualizar la página para comprobar si Airflow está bien reiniciado. Ok, Airflow se está ejecutando vamos a hacer nuestra prueba. Haga clic en el toggle del dag parallel_dag para programar (schedule) el DAG y empezar a actualizar la página. Como ya puede ver por el círculo aquí, se han iniciado múltiples DAGRuns ya que el Scheduler está rellenando (backfilling) todas las instancias DagRun para las que no tiene ningún estado registrado en la metadata database, a partir de la start_date definida en el DAG. 

<center><img src="https://i.postimg.cc/T1s9vvfv/a781.png"></center>

Si haces clic en el DAG y actualizas la página varias veces, observarás un comportamiento interesante de Airflow. Eche un vistazo al orden en que se ejecutan las tareas de cada DAGRun. ¿Encuentra algo extraño? 

<center><img src="https://i.postimg.cc/j5ZcvJ15/a782.png"></center>

Podríamos haber enseñado que Airflow dispara cada DAG Run de uno en uno y espera a que termine el primero para empezar el siguiente, pero no es así en absoluto. En realidad, inicia las tareas por nivel de dependencias. En nuestro caso, se disparan las tres primeras tareas de cada DAGRun, luego la tarea 4 y por último la tarea 5. Además, se puede ver que sólo dos tareas se ejecutan en paralelo por DAGRun como se muestra aquí. 

<center><img src="https://i.postimg.cc/Wzzny1R7/a783.png"></center>

Si actualizas la página varias veces, siempre tenemos dos tareas ejecutándose en paralelo tal y como lo definimos en el parámetro dag_concurrency. Por último, observa que Airflow deja de programar nuevos DAGRuns después de alcanzar el 16º. 

<center><img src="https://i.postimg.cc/3Jy1CNF8/a784.png"></center>

¿Por qué? Porque si vamos a nuestro editor y abrimos el fichero airflow.cfg, el parámetro max_active_runs_per_dag está configurado por defecto a 16 lo que significa que podemos tener hasta 16 DAGRuns activos por DAG. 

<center><img src="https://i.postimg.cc/NfW4Ntd1/a785.png"></center>

Cambie su valor por 1 y abra el terminal. 

<center><img src="https://i.postimg.cc/SxCG3PPH/a786.png"></center>

Desde allí, reinicie Airflow con los mismos comandos que hicimos antes y vuelva al navegador web. Actualicemos la página. Ok, ahora programe (schedule) el DAG de nuevo. Actualice la página y haga clic en el DAG. Creo que ya puedes ver la diferencia. Ahora sólo tenemos un DAGRun y no 16 ejecutándose al mismo tiempo. 

<center><img src="https://i.postimg.cc/s2zJrcdx/a787.png"></center>

Ahora te preguntarás si es mejor poner este parámetro a 1 o a 16. Bueno, si hay una dependencia entre tus DAGRuns, como los datos producidos por un DAGRun son usados por el siguiente DAGRun, entonces definitivamente debes poner este parámetro a 1. Si no, puedes mantenerlo a 1. Si no, puedes mantenerlo al valor por defecto. Sólo ten en cuenta que este parámetro es muy importante cuando se utiliza backfilling. Muy bien, antes de pasar al siguiente video, vuelve a tu terminal y detén el contenedor docker. 

<center><img src="https://i.postimg.cc/DzcPyXng/a788.png"></center>

Así que espero que hayas aprendido mucho durante este video, no dudes en hacer tus propios experimentos con estos parámetros es realmente importante entender.